Este notebook está vinculado al caso de estudio "Yapo" (ver el [documento](https://docs.google.com/document/d/1SwhwMOyG7-cmzasDeisQZ_NzX5v2VoZ7--lI4UhJz_U/edit#heading=h.5qaqs0frqkx4) de presentación). Se busca construir una base de datos de anuncios en Los Rios recopilando algunos anuncios de Yapo.cl.


# 1. Construcción de la base de datos

In [1]:
import mysql.connector 

db_connection = mysql.connector.connect(user="root",host="localhost",password="root")
cursor = db_connection.cursor()

In [11]:
cursor.execute("DROP DATABASE Yapo2;")

In [12]:
cursor.execute("CREATE DATABASE Yapo2;")
cursor.execute("USE Yapo2")

#tabla vendedor
cursor.execute("CREATE TABLE vendedor ("+
               "id_vendedor VARCHAR(130) PRIMARY KEY, nombre VARCHAR(100), ciudad VARCHAR(30), codigo_region VARCHAR(7), "+
               "nombre_region VARCHAR(30), "+
               "fecha_inscripcion DATE)")

#tabla anuncio
cursor.execute("CREATE TABLE anuncio (url VARCHAR(300) PRIMARY KEY, "+
               "titulo VARCHAR(200), descripcion MEDIUMTEXT, precio INT, moneda VARCHAR(2), fecha_publicacion DATE, categoria VARCHAR(50), "
               +"id_vendedor VARCHAR(130), FOREIGN KEY (id_vendedor) REFERENCES vendedor(id_vendedor))")

# 2. Scrapping de datos para llenar nuestra base de datos Yapo

In [4]:
from urllib.request import urlopen as uReq
import urllib.request

from bs4 import BeautifulSoup as soup
from selenium import webdriver
import time

Con la librería Selenium, abriremos un navegador Chrome, en la página de los anuncios de Yapo en la región de Los Rios... __(UPDATE: todo chile)__

In [13]:
browser = webdriver.Chrome()

myUrl = 'https://www.yapo.cl/chile/todos_los_avisos?ca=12_s&l=0'
#myUrl = 'https://www.yapo.cl/los_rios/todos_los_avisos?ca=11_s&l=0&w=1&cmn=243'
browser.get(myUrl)

Con la librería BeautifulSoup, realizamos un scrapping del código HTML para recuperar el enlace de la "última página"...

In [14]:
pageSoup = soup(browser.page_source, 'html.parser')

pages = pageSoup.find('span',  {'class', 'nohistory FloatRight'}).a['href']

In [15]:
pages

'https://www.yapo.cl/chile?ca=12_s&o=28975'

El parametro 'o' de la URL corresponde al número de la pagina en la lista de anuncios.

In [16]:
index = pages.rfind('=')
print(index)
lastPage = int(pages[index+1:])
print(lastPage)
root_pages = pages[:index+1]
print(root_pages)

35
28975
https://www.yapo.cl/chile?ca=12_s&o=


- la variable "index" permite identificar el indice del último caracter '='.

- la variable "lastPage" permite identificar el numero de la última página en la URL.

- la variable "root_pages" permite aislar la cadena de caracteres que corresponde a la raiz de la URL (sin el numero de página).


Empezaremos la iteración sobre cada página que escrapear...

In [17]:
import datetime

def convert_date(yapo_date):
    month=yapo_date.split(" ")[0]
    year=yapo_date.split(" ")[1]
    
    convert = {'Enero' : 1,
            'Febrero' : 2,
            'Marzo' : 3,
            'Abril' : 4,
            'Mayo' : 5,
            'Junio' : 6,
            'Julio' : 7,
            'Agosto' : 8,
            'Septiembre' : 9, 
            'Octubre' : 10,
            'Noviembre' : 11,
            'Diciembre' : 12
           }
    
    
    new_date=datetime.date(int(year), convert[month], 1) 

    return new_date

print(convert_date("Febrero 2020"))

2020-02-01


In [ ]:
for i in range(lastPage):
    
    #recuperarmos la URL de la pagina corriente
    url = root_pages + str(i+1)
    
    #con Selenium, vamos en navegar en esta página
    browser.get(url)
    
    #empezamos el scrapping de la pagina corriente
    pageSoup = soup(browser.page_source, 'html.parser')
    
    #recuperamos todos los tags HTML que corresponden a la lista de anuncios en esta pagina
    links = pageSoup.findAll('td', {'class' : 'thumbs_subject'})
    
    #empezamos a iterar sobre cada anuncio
    for link in links:
        
        #todos los datos que necesitamos encontrar
        url, titulo, descripcion, precio, fecha_publicacion, categoria="","","",None,"", ""
        nombre, ciudad, codigo_region, nombre_region, fecha_inscripcion="","","","",""
        
        #Navegamos hacia la pagina del anuncio
        url=link.find('a',{'class':'title'})['href']
        print(url)
        browser.get(link.find('a',{'class':'title'})['href'])

        #RECUPERAMOS LA FECHA DE PUBLICACION DEL ANUNCIO          
        pageSoup = soup(browser.page_source, 'html.parser')
        if(pageSoup.find('time')):
            datetime_raw = pageSoup.find('time').attrs['datetime']
            date_publication_raw=datetime_raw.split("T")[0]
            date_publication_raw=  date_publication_raw.split("-")
            
            date_publication=datetime.date(int(date_publication_raw[0]), int(date_publication_raw[1]),
                                           int(date_publication_raw[2]))
            print("datetime:")
            print(date_publication)
        
        #RECUPERAMOS EL TITULO DEL ANUNCIO           
        pageSoup = soup(browser.page_source, 'html.parser')
        if(pageSoup.find('h1', {"id" : "da_subject"})):
            titulo = pageSoup.find('h1', {"id" : "da_subject"}).text.strip()
            print(titulo)
            
        #RECUPERAMOS LA DESCRIPCION DEL ANUNCIO
        if(pageSoup.find('div', {"class" : "description"})):
            try:
                descripcion = pageSoup.find('div', {"class" : "description"}).text.split(' ', 1)[1].strip().replace(u'\n', u' ')
            except:
                continue
                
        #RECUPERAMOS EL PRECIO DEL ANUNCIO
        if(pageSoup.find('div', {"class" : "price text-right"})):
            precio_raw = pageSoup.find('div', {"class" : "price text-right"}).text.strip().replace(u'\n', u' ').replace(u'\t', u'')
            precio_raw = precio_raw.split(" ")
            moneda = precio_raw[0]
            precio = precio_raw[1].split(",")[0].replace(u'.', u'')
            print("precio:"+precio)
            
        #RECUPERAMOS LA CATEGORIA DEL ANUNCIO
        if(pageSoup.find('div', {"class" : "breadcrumbs"})):
            categoria = pageSoup.find('div', {"class" : "breadcrumbs"}).find('a', {"id" : "breadcrumb_category"}).find('strong').text.strip().replace(u'\n', u' ')
            print(categoria)
            
        
        #RECUPERAMOS EL NOMBRE DEL VENDEDOR, SU FECHA DE INSCRIPCION Y SU LOCALIDAD
        if(pageSoup.find('aside', {"class" : "sidebar-right"})):
            aside = pageSoup.find('aside', {"class" : "sidebar-right"})
    
            #print(aside.find('seller-info'))
            
            #NOMBRE
            if(aside.find('seller-info')!=None):
                nombre=aside.find('seller-info').attrs['username']
            
                #FECHA DE INSCRIPCION
                fecha_inscripcion_raw=aside.find('seller-info').attrs['seniority']
                try:
                    fecha_inscripcion_raw=fecha_inscripcion_raw[len("En Yapo desde "):]
                    fecha_inscripcion=convert_date(fecha_inscripcion_raw)
                    print(fecha_inscripcion)
                except:
                    fecha_inscripcion=None
                    continue
                
                #LOCALIDAD
                localidad_raw=aside.find('seller-info').attrs['region']
            
                #print(localidad_raw)
            
                region_raw=localidad_raw.split(",")[0]
                ciudad_raw=localidad_raw.split(",")[1]
            
                codigo_region=region_raw.split(" ")[0]
                nombre_region=region_raw[len(codigo_region)+1:]
                ciudad=ciudad_raw[1:]
                
        # LLENAMOS LA BASE DE DATOS: TABLA VENDEDOR
        
        try:
            sql = "INSERT INTO vendedor (id_vendedor,nombre, ciudad, codigo_region, nombre_region, fecha_inscripcion) VALUES (%s,%s, %s, %s,%s,%s)"
            val = (nombre+"_"+codigo_region+"_"+ciudad,nombre, ciudad, codigo_region, nombre_region, fecha_inscripcion)
            cursor.execute(sql, val)
        except Exception as e1:
            print(e1)
            #continue
        
        # LLENAMOS LA BASE DE DATOS: TABLA ANUNCIO
        try:
            sql = "INSERT INTO anuncio (url, titulo, descripcion, precio, moneda, fecha_publicacion, categoria, id_vendedor) VALUES (%s, %s, %s,%s, %s, %s, %s, %s)"
            val = (url, titulo, descripcion, int(precio), moneda, date_publication, categoria, nombre+"_"+codigo_region+"_"+ciudad)
            cursor.execute(sql, val)
        except Exception as e2:
            print(e2)
            #continue
        
        cursor.execute("COMMIT")

https://www.yapo.cl/region_metropolitana/moda-vestuario/chaqueta_cuero_unisex_73859055.htm?ca=15_s&first=1&oa=73859055&xsp=0
datetime:
2020-08-19
Chaqueta cuero unisex
precio:20000
Moda y vestuario
2014-09-01
https://www.yapo.cl/ohiggins/ofertas_de_empleo/asistente_informatica_73859054.htm?ca=7_s&oa=73859054&xsp=1
datetime:
2020-08-19
Asistente Informática
precio:500000
Ofertas de empleo
2017-02-01
https://www.yapo.cl/magallanes_antartica/salud-belleza/compro_ensure_advance_y_glucerna_73859053.htm?ca=14_s&oa=73859053&xsp=2
datetime:
2020-08-19
Compro Ensure advance y glucerna
Salud y belleza
2014-11-01
int() argument must be a string, a bytes-like object or a number, not 'NoneType'
https://www.yapo.cl/region_metropolitana/accesorios_vehiculos/moldura_chevrolet_spark_lt_73859052.htm?ca=15_s&oa=73859052&xsp=3
datetime:
2020-08-19
Moldura chevrolet spark lt
precio:15000
Accesorios y piezas para vehículos
2015-08-01
https://www.yapo.cl/region_metropolitana/ofertas_de_empleo/decoradora_de_t

https://www.yapo.cl/region_metropolitana/arrendar/comodo_dpto__2d_1b_e__metro_sta__lucia_73859023.htm?ca=15_s&oa=73859023&xsp=36
datetime:
2020-08-19
Cómodo dpto. 2D/1B/E. Metro Sta. Lucía
precio:380000
Arriendo - Departamento
2019-08-01
https://www.yapo.cl/region_metropolitana/arrendar/depto_amplio_1dorm__living_metro_parque_bustamante_73859022.htm?ca=15_s&oa=73859022&xsp=37
datetime:
2020-08-19
Depto.Amplio 1Dorm. living Metro parque Bustamante
precio:300000
Arriendo - Departamento
2014-05-01
https://www.yapo.cl/region_metropolitana/consolas_videojuegos/mortal_kombat_arcade_73859020.htm?ca=15_s&oa=73859020&xsp=38
datetime:
2020-08-19
Mortal kombat arcade
precio:500000
Consolas, videojuegos y accesorios
2014-12-01
https://www.yapo.cl/region_metropolitana/moda-vestuario/chaqueta_americanino_con_bordado_73859019.htm?ca=15_s&oa=73859019&xsp=39
datetime:
2020-08-19
Chaqueta AMERICANINO con bordado
precio:13000
Moda y vestuario
2017-06-01
https://www.yapo.cl/region_metropolitana/moda-vestu

https://www.yapo.cl/region_metropolitana/busco_empleo/cuidado_de_adulto_mayor_73858992.htm?ca=15_s&oa=73858992&xsp=19
datetime:
2020-08-19
Cuidado de adulto mayor
Busco empleo
2020-01-01
int() argument must be a string, a bytes-like object or a number, not 'NoneType'
https://www.yapo.cl/magallanes_antartica/salud-belleza/kit_de_maquillaje_y_levanta_pestanas_73858991.htm?ca=14_s&oa=73858991&xsp=20
datetime:
2020-08-19
Kit de maquillaje y levanta pestañas
precio:1990
Salud y belleza
2020-08-01
1062 (23000): Duplicate entry 'Juan alvarez_XII_Punta Arenas' for key 'PRIMARY'
https://www.yapo.cl/region_metropolitana/articulos-del-hogar/moledor_molinillo_electrico_de_cafe_73858990.htm?ca=15_s&oa=73858990&xsp=21
datetime:
2020-08-19
Moledor Molinillo Electrico De Café
precio:9990
Otros artículos del hogar
2020-08-01
1062 (23000): Duplicate entry 'Luis moran_Región_Santiago' for key 'PRIMARY'
https://www.yapo.cl/region_metropolitana/calzado/zapato_mtb_shimano_talla_44_seminuevo_73858989.htm?ca=

https://www.yapo.cl/magallanes_antartica/accesorios_vehiculos/manos_libres_y_cargador_auto_73859098.htm?ca=14_s&oa=73859098&xsp=0
datetime:
2020-08-19
Manos libres y cargador auto
Accesorios y piezas para vehículos
2020-08-01
1062 (23000): Duplicate entry 'Juan alvarez_XII_Punta Arenas' for key 'PRIMARY'
int() argument must be a string, a bytes-like object or a number, not 'NoneType'
https://www.yapo.cl/magallanes_antartica/television_camaras/audifonos_sony_portatil_color_negro_73859097.htm?ca=14_s&oa=73859097&xsp=1
datetime:
2020-08-19
Audifonos sony portatil color negro
precio:9990
Audio, TV, video y fotografía
2020-08-01
1062 (23000): Duplicate entry 'Juan alvarez_XII_Punta Arenas' for key 'PRIMARY'
https://www.yapo.cl/magallanes_antartica/celulares/lente_telescopio_8x_zoom_para_tlf_celular_smartpho_73859096.htm?ca=14_s&oa=73859096&xsp=2
datetime:
2020-08-19
Lente telescopio 8x zoom para tlf celular smartpho
Celulares, teléfonos y accesorios
2020-08-01
1062 (23000): Duplicate entry 

datetime:
2020-08-19
Memoria Ram Crucial 8gb( 2x4gb) 3200 Mhz
precio:49990
Computadores y accesorios
2020-02-01
https://www.yapo.cl/region_metropolitana/arrendar/arriendo_local_comercial_galeria_alessandri_73736716.htm?ca=15_s&oa=73736716&xsp=30
datetime:
2020-08-19
Arriendo Local Comercial Galería Alessandri
precio:2700000
Arriendo - Comercial e industrial
2017-09-01
https://www.yapo.cl/magallanes_antartica/salud-belleza/cepillo_alisador_de_cabello_color_rojo_73859070.htm?ca=14_s&oa=73859070&xsp=31
datetime:
2020-08-19
Cepillo alisador de cabello color rojo
precio:7990
Salud y belleza
2020-08-01
1062 (23000): Duplicate entry 'Juan alvarez_XII_Punta Arenas' for key 'PRIMARY'
https://www.yapo.cl/region_metropolitana/accesorios_vehiculos/tapas_citroen_picasso_73859069.htm?ca=15_s&oa=73859069&xsp=32
datetime:
2020-08-19
Tapas citroen picasso
precio:20000
Accesorios y piezas para vehículos
2015-08-01
1062 (23000): Duplicate entry 'alejandro isla _Región_Puente Alto' for key 'PRIMARY'
https

...

datetime:
2020-08-19
Cardan Chevrolet s10
precio:45000
Accesorios y piezas para vehículos
2015-02-01
1062 (23000): Duplicate entry 'German_Región_Buin' for key 'PRIMARY'
https://www.yapo.cl/ohiggins/vestuario-futura-mama-ninos/calza_nina_talla_8_73858184.htm?ca=7_s&oa=73858184&xsp=2
datetime:
2020-08-19
Calza niña talla 8
precio:2000
Vestuario futura mamá y niños
2016-09-01
1062 (23000): Duplicate entry 'Monica_VI_Rancagua' for key 'PRIMARY'
https://www.yapo.cl/region_metropolitana/accesorios_vehiculos/alza_vidrio_chevrolet_blazer_s10_73858183.htm?ca=15_s&oa=73858183&xsp=3
datetime:
2020-08-19
Alza vidrio Chevrolet blazer s10
precio:35000
Accesorios y piezas para vehículos
2015-02-01
1062 (23000): Duplicate entry 'German_Región_Buin' for key 'PRIMARY'
https://www.yapo.cl/ohiggins/vestuario-futura-mama-ninos/calza_colloky_nina_talla_8_73858182.htm?ca=7_s&oa=73858182&xsp=4
datetime:
2020-08-19
Calza colloky niña talla 8
precio:3500
Vestuario futura mamá y niños
2016-09-01
1062 (23000): D

datetime:
2020-08-19
Calcetas largas con mariposas
precio:3000
Moda y vestuario
2014-02-01
1062 (23000): Duplicate entry 'cristobal arriagada_Región_Ñuñoa' for key 'PRIMARY'
https://www.yapo.cl/region_metropolitana/calzado/adidas_yeezy_boost_350_v2_73858159.htm?ca=15_s&oa=73858159&xsp=28
datetime:
2020-08-19
Adidas YEEZY BOOST 350 V2
precio:79990
Calzado
2014-02-01
1062 (23000): Duplicate entry 'cristobal arriagada_Región_Santiago' for key 'PRIMARY'
https://www.yapo.cl/region_metropolitana/servicios/cedula_identidad_para_mascota_73858158.htm?ca=15_s&oa=73858158&xsp=29
datetime:
2020-08-19
Cedula identidad para mascota
precio:5000
Servicios
2014-02-01
1062 (23000): Duplicate entry 'oriana lagos alvarado_Región_Recoleta' for key 'PRIMARY'
https://www.yapo.cl/region_metropolitana/moda-vestuario/polera_corta_tigre_73858157.htm?ca=15_s&oa=73858157&xsp=30
datetime:
2020-08-19
Polera corta tigre
precio:8990
Moda y vestuario
2014-02-01
1062 (23000): Duplicate entry 'cristobal arriagada_Región_

datetime:
2020-08-19
M2 Carpintero
Ofertas de empleo
2018-11-01
1062 (23000): Duplicate entry 'A visos_VIII_Coronel' for key 'PRIMARY'
int() argument must be a string, a bytes-like object or a number, not 'NoneType'
https://www.yapo.cl/valparaiso/moda-vestuario/joggers_caqui_73858133.htm?ca=6_s&oa=73858133&xsp=5
datetime:
2020-08-19
Joggers caqui
precio:18000
Moda y vestuario
2018-12-01
1062 (23000): Duplicate entry 'Josefa atalah torrealba_V_Villa Alemana' for key 'PRIMARY'
https://www.yapo.cl/araucania/comprar/casa_puerto_saavedra_2_habitaciones_73858132.htm?ca=10_s&oa=73858132&xsp=6
datetime:
2020-08-19
Casa puerto saavedra 2 habitaciones
precio:22000000
Vendo - Casa
2014-01-01
1062 (23000): Duplicate entry 'enrique_IX_Saavedra' for key 'PRIMARY'
https://www.yapo.cl/araucania/electrodomesticos/estufa_a_gas_con_cilindro_de_11_73858131.htm?ca=10_s&oa=73858131&xsp=7
datetime:
2020-08-19
Estufa a gas con cilindro de 11
precio:65000
Electrodomésticos
2020-08-01
1062 (23000): Duplicate en

datetime:
2020-08-19
Iphone 11 128 gb blanco
Celulares, teléfonos y accesorios
2017-02-01
1062 (23000): Duplicate entry 'jose lopez_VIII_San Pedro de la Paz' for key 'PRIMARY'
int() argument must be a string, a bytes-like object or a number, not 'NoneType'
https://www.yapo.cl/region_metropolitana/deportes_gimnasia/fitbit_charge_3__73858106.htm?ca=15_s&oa=73858106&xsp=33
datetime:
2020-08-19
Fitbit charge 3
Deportes, gimnasia y accesorios
2017-09-01
1062 (23000): Duplicate entry 'isidora _Región_Las Condes' for key 'PRIMARY'
int() argument must be a string, a bytes-like object or a number, not 'NoneType'
https://www.yapo.cl/valparaiso/autos/sanyong_korando_2_9_73858105.htm?ca=6_s&oa=73858105&xsp=34
datetime:
2020-08-19
Sanyong korando 2.9
precio:5000000
Autos, camionetas y 4x4
2014-04-01
1062 (23000): Duplicate entry 'Paolo Zelaya_V_Los Andes' for key 'PRIMARY'
https://www.yapo.cl/los_rios/television_camaras/tv_73858104.htm?ca=11_s&oa=73858104&xsp=35
datetime:
2020-08-19
TV
precio:15000

https://www.yapo.cl/valparaiso/salud-belleza/rollo_suave_para_pies_73858078.htm?ca=6_s&oa=73858078&xsp=11
datetime:
2020-08-19
Rollo suave para pies
precio:3000
Salud y belleza
2015-06-01
1062 (23000): Duplicate entry 'Carolina_V_Viña del Mar' for key 'PRIMARY'
https://www.yapo.cl/region_metropolitana/autos/hyundai_accent_2003_73858077.htm?ca=15_s&oa=73858077&xsp=12
datetime:
2020-08-19
Hyundai accent 2003
precio:2400000
Autos, camionetas y 4x4
2015-09-01
1062 (23000): Duplicate entry 'claudio mendoza_Región_Isla de Maipo' for key 'PRIMARY'
https://www.yapo.cl/region_metropolitana/comprar/departamento_el_olivo_b___region_metropolit__8230__73858076.htm?ca=15_s&oa=73858076&xsp=13
datetime:
2020-08-19
DEPARTAMENTO El Olivo B - Región Metropolit…
precio:35000000
Vendo - Departamento
2018-09-01
1062 (23000): Duplicate entry 'Agente Propiedades_Región_San Bernardo' for key 'PRIMARY'
https://www.yapo.cl/coquimbo/computadores/multifuncional__73858075.htm?ca=5_s&oa=73858075&xsp=14
datetime:
202

In [ ]:
#¿Insertar una columna “Perfil” en la tabla Vendedor. 
#Los Vendedores que publicaron más de 5 anuncios en los últimos 7 días, tienen un perfil “Pro”
#, los otros se consideran “Personal”.

sql = "alter table vendedor add perfil varchar(10);"
cursor.execute(sql)

sql = "select * from vendedor;"
cursor.execute(sql)

vendedor = DataFrame(cursor.fetchall())
df1

In [ ]:
...